In [1]:
import pandas as pd
import numpy as np
import jiwer
import glob

In [4]:
xlsx_files = glob.glob('./*.xlsx')
print(len(xlsx_files))

12


### Normalization

In [5]:
import sys
import os
current = os.path.dirname(os.path.realpath(__name__))
parent = os.path.dirname(current)
sys.path.append(parent)
from Normalizer import filterAndNormalize, normalizeOnly

/scratch/junzisun/env/jan/lib/python3.10/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


### WER Insights

In [6]:
def calcWERInfo(ref, hyp):
    out = jiwer.process_words(ref, hyp)
    return jiwer.visualize_alignment(out)

def calcWER(ref, hyp):
    return jiwer.wer(ref, hyp)*100

In [7]:
for file in xlsx_files:
    df = pd.read_excel(file)
    df = df.drop(columns=['Unnamed: 0'])
    
    df['ref-norm'] = df.apply(lambda x: filterAndNormalize(str(x['ref'])), axis=1)
    df['hyp-clean-norm'] = df.apply(lambda x: filterAndNormalize(str(x['hyp-clean'])), axis=1)
    df['hyp-prmpt-norm'] = df.apply(lambda x: filterAndNormalize(str(x['hyp-prmpt'])), axis=1)
    
    df['WER-info-clean'] = df.apply(lambda x: calcWERInfo(str(x['ref']), str(x['hyp-clean'])), axis=1)
    df['WER-info-prmpt'] = df.apply(lambda x: calcWERInfo(str(x['ref']), str(x['hyp-prmpt'])), axis=1)
    df['WER-info-clean-norm'] = df.apply(lambda x: calcWERInfo(str(x['ref-norm']), str(x['hyp-clean-norm'])), axis=1)
    df['WER-info-clean-prmpt'] = df.apply(lambda x: calcWERInfo(str(x['ref-norm']), str(x['hyp-prmpt-norm'])), axis=1)
    
    df['WER-clean'] = df.apply(lambda x: calcWER(str(x['ref']), str(x['hyp-clean'])), axis=1)
    df['WER-prmpt'] = df.apply(lambda x: calcWER(str(x['ref']), str(x['hyp-prmpt'])), axis=1)
    df['WER-clean-norm'] = df.apply(lambda x: calcWER(str(x['ref-norm']), str(x['hyp-clean-norm'])), axis=1)
    df['WER-clean-prmpt'] = df.apply(lambda x: calcWER(str(x['ref-norm']), str(x['hyp-prmpt-norm'])), axis=1)
    
    for s in ['train+validation', 'validation']:	
        df3 = df[df['split'].isin(s.split('+'))]
        
        wer_avg_clean = calcWER(list(df3['ref']), list(df3['hyp-clean']))
        wer_avg_prmpt = calcWER(list(df3['ref']), list(df3['hyp-prmpt']))
        wer_avg_clean_norm = calcWER(list(df3['ref-norm']), list(df3['hyp-clean-norm']))
        wer_avg_prmpt_norm = calcWER(list(df3['ref-norm']), list(df3['hyp-prmpt-norm']))
        
        df.loc['avg-'+s, ['WER-clean', 'WER-prmpt', 'WER-clean-norm', 'WER-clean-prmpt']] = [round(wer_avg_clean, 4), round(wer_avg_prmpt, 4), round(wer_avg_clean_norm, 4), round(wer_avg_prmpt_norm, 4)]
        
    df.to_excel(file[:-5]+'-WER_insights.xlsx')